In [19]:
from pathlib import Path

import numpy as np
import pandas as pd
import geopandas as gpd
from shapely import wkb, wkt
from shapely.geometry import box, Polygon

import rasterio
import rioxarray as rxa

import matplotlib.pyplot as plt

from datetime import datetime

from metloom.pointdata import SnotelPointData, CDECPointData
from metloom.variables import SnotelVariables, CdecStationVariables

from rasterio import features
def vectorize_valid(fp):
    img = rxa.open_rasterio(fp).squeeze('band')
    img = ~img.isnull()
    img = img.astype('uint8')
    mask = img == 1
    coords = list((features.shapes(img, mask = mask)))[0][0]['coordinates'][0]
    xy_coords = [(img.x[int(x)].values.ravel()[0], img.y[int(y)].values.ravel()[0]) for x, y in coords]
    return Polygon(xy_coords)

fig_dir = Path('/bsuhome/zacharykeskinen/uavsar-coherence/figures/snotels')

uavsar_dir = Path('/bsuhome/zacharykeskinen/scratch/coherence/uavsar')

uavsars = list(uavsar_dir.glob('*'))
uavsars = [u for u in uavsars if u != 'tmp']
uavsars = uavsars[:2]
uavsars = {'_'.join(u.stem.split('_')[:2]): list(u.glob('*.cor.grd.tiff'))[0] for u in uavsars if len(list(u.glob('*.cor.grd.tiff'))) > 0}
bounds = {u: vectorize_valid(v) for u, v in uavsars.items()}
gpd.GeoDataFrame(bounds.values(), index = bounds.keys(), columns = ['geometry'])

,geometry
grmesa_27416,"POLYGON ((-108.25885 39.16227, -108.25880 39.1..."
stlake_09127,"POLYGON ((-111.68318 40.67539, -111.68241 40.6..."
